In [1]:
import numpy as np
import networkx as nx
import random

In [100]:
pts = np.array([np.random.uniform([-20, -20, 0], [20, 20, 5]) for _ in range(50)])
G = nx.Graph()
for i in range(len(pts)):
    G.add_node(i, pos=tuple(pts[i]))

for i in range(len(pts)):
        for j in range(i+1, len(pts)):
            if np.random.rand() < 0.3:
                distance = np.linalg.norm(pts[i] - pts[j])
                G.add_edge(i, j, distance=distance)

In [90]:
A = nx.adjacency_matrix(G, weight='distance')

In [101]:
elevations = [G.nodes[i]['pos'][2] for i in G.nodes]
treatment_idxs = np.argsort(elevations)[:5]


In [102]:
for i in G.nodes:
    if i in treatment_idxs:
        G.nodes[i]['treatment'] = 1
    else:
        G.nodes[i]['treatment'] = 0

In [103]:
treatment_nodes = [i for i in G.nodes if G.nodes[i]['treatment'] == 1]
source_nodes = [i for i in G.nodes if G.nodes[i]['treatment'] == 0]

In [104]:
print(f"# Source: {len(source_nodes)}; # treatment: {len(treatment_nodes)}; # Components = {nx.number_connected_components(G)}")

Path = {}
NLinks = {}
L = {}

# Source: 45; # treatment: 5; # Components = 1


In [76]:
def make_connected(G):
    # Get the list of connected components
    components = list(nx.connected_components(G))
    
    # If the graph is already connected, return it as is
    if len(components) == 1:
        return G
    
    # Create a copy of the graph to modify
    H = G.copy()
    
    while len(components) > 1:
        # Choose two random components
        c1, c2 = random.sample(components, 2)
        
        # Choose a random node from each component
        n1 = random.choice(list(c1))
        n2 = random.choice(list(c2))
        
        # Add an edge between these nodes
        H.add_edge(n1, n2)
        
        # Recalculate the connected components
        components = list(nx.connected_components(H))
    
    return H

In [85]:
# G = make_connected(G)

In [106]:
for i in source_nodes:
    for j in treatment_nodes:
        path = nx.shortest_path(G, source=i, target=j, weight='distance')
        Path[i, j] = path
        NLinks[i, j] = len(path)
        L[i, j] = nx.path_weight(G, Path[i, j], weight='distance')

In [110]:
d = {1: 1, 2:2, 3:3}
sum(d.values())

6

In [112]:
G.nodes[1]

{'pos': (6.932977269549955, 12.408448411736629, 4.375633189763651),
 'treatment': 0}

In [115]:
LE = {e:G.edges[e]['distance'] for e in G.edges}

{(0, 1): 21.762631136054566,
 (0, 6): 30.19036008540191,
 (0, 9): 5.978530445736753,
 (0, 10): 29.146889653741034,
 (0, 12): 34.28221236709288,
 (0, 20): 32.187193182004485,
 (0, 21): 31.906027027110646,
 (0, 23): 36.18500925927551,
 (0, 25): 18.846463938214356,
 (0, 36): 0.8449901282600514,
 (0, 37): 28.277278637294884,
 (0, 40): 32.61622131015464,
 (0, 42): 6.65183999628691,
 (0, 44): 30.30286937827982,
 (1, 2): 23.71393408909411,
 (1, 6): 9.942016965040038,
 (1, 11): 35.28509104075213,
 (1, 13): 25.073801986085808,
 (1, 27): 3.826904485148119,
 (1, 28): 19.10030314627021,
 (1, 30): 23.96751779050768,
 (1, 31): 24.897561257797495,
 (1, 33): 11.442311293696376,
 (1, 36): 22.55975339535601,
 (1, 40): 40.04469693594184,
 (1, 44): 10.376402659067063,
 (2, 5): 25.980188997277445,
 (2, 11): 35.20869022917933,
 (2, 15): 20.001749575467983,
 (2, 16): 4.597069569195431,
 (2, 24): 31.193628250099902,
 (2, 29): 40.3981699871247,
 (2, 31): 16.06499954057905,
 (2, 40): 36.155209901969805,
 (2, 41

In [ ]:


def connect_nearest_neighbors(G):
  """Connects each node to its three nearest neighbors based on 3D positions.

  Args:
    G: A NetworkX graph with 'pos' attributes for each node.

  Returns:
    The modified graph with added edges.
  """

  node_positions = nx.get_node_attributes(G, 'pos')
  node_list = list(G.nodes())

  for node in node_list:
    distances = []
    for neighbor in node_list:
      if node != neighbor:
        distance = np.linalg.norm(np.array(node_positions[node]) - np.array(node_positions[neighbor]))
        distances.append((neighbor, distance))

    # Sort distances in ascending order
    distances.sort(key=lambda x: x[1])

    # Add edges to the three nearest neighbors
    for i in range(3):
      G.add_edge(node, distances[i][0])

  return G